In [31]:
import numpy as np
from scipy.special import logsumexp, betaln, gammaln
from scipy.stats import norm

def pt_d_sample_test(X, Y, c=1, max_depth=-1, qdist=norm.ppf, aj=lambda depth: depth**2, log_BF=False):
    old_expressions = np.get_printoptions()['threshold']
    np.set_printoptions(threshold=max(max_depth, old_expressions))

    if max_depth < 0:
        max_depth = max(1, int(np.floor(np.log2(len(X)) / 2)))

    binary, continuous = (X, Y) if is_discrete(X) else (Y, X)
    data = np.column_stack([scale(continuous), binary])
    X = data[:, 0]

    p_H0 = pt_marginal_likelihood(X, low=0, up=1, c=c, depth=1, max_depth=max_depth, qdist=qdist, aj=aj)

    discrete_values = binary if len(np.unique(binary)) == 2 else np.unique(binary)

    p_H1 = max([pt_marginal_likelihood(data[data[:, 1] == i, 0], low=0, up=1, c=c, depth=1, max_depth=max_depth, qdist=qdist, aj=aj) +
                pt_marginal_likelihood(data[data[:, 1] != i, 0], low=0, up=1, c=c, depth=1, max_depth=max_depth, qdist=qdist, aj=aj)
                for i in discrete_values])

    n_hypotheses = len(discrete_values)
    bf = p_H0 - p_H1 + np.log(n_hypotheses)   #correction

    np.set_printoptions(threshold=old_expressions)

    if log_BF:
        return {'bf': bf, 'p_H0': None, 'p_H1': None}

    bf = np.exp(bf)
    return {'bf': bf, 'p_H0': 1 - 1 / (1 + bf), 'p_H1': 1 / (1 + bf)}

def pt_marginal_likelihood(data, low, up, c, depth, max_depth, qdist, aj):
    if depth == max_depth:
        return 0

    if isinstance(low, (int, float)):     #if low is an integer
        n_j = [
            np.sum((qdist(low) < data) & (data <= qdist((low + up) / 2))),
            np.sum((qdist((low + up) / 2) < data) & (data <= qdist(up)))
        ]           # counts the number of data in each interval (2 vector).
    else:
        n_j = [
            np.sum((qdist(low[0]) < data[:, 0]) & (data[:, 0] <= qdist((low[0] + up[0]) / 2)) &
                   (qdist(low[1]) < data[:, 1]) & (data[:, 1] <= qdist((low[1] + up[1]) / 2))),
            np.sum((qdist((low[0] + up[0]) / 2) < data[:, 0]) & (data[:, 0] <= qdist(up[0])) &
                   (qdist(low[1]) < data[:, 1]) & (data[:, 1] <= qdist((low[1] + up[1]) / 2))),
            np.sum((qdist(low[0]) < data[:, 0]) & (data[:, 0] <= qdist((low[0] + up[0]) / 2)) &
                   (qdist((low[1] + up[1]) / 2) < data[:, 1]) & (data[:, 1] <= qdist(up[1]))),
            np.sum((qdist((low[0] + up[0]) / 2) < data[:, 0]) & (data[:, 0] <= qdist(up[0])) &
                   (qdist((low[1] + up[1]) / 2) < data[:, 1]) & (data[:, 1] <= qdist(up[1])))
        ]     #4 vector

    if np.sum(n_j) == 0:
        return 0

    a_j = c * aj(depth)

    if len(n_j) == 2:
        logl = betaln(n_j[0] + a_j, n_j[1] + a_j) - betaln(a_j, a_j)      #log of beta function
    else:
        logl = lmbeta(n_j[0] + a_j, n_j[1] + a_j, n_j[2] + a_j, n_j[3] + a_j) - lmbeta(a_j, a_j, a_j, a_j)

    if isinstance(low, (int, float)):
        likelihoods = [
            pt_marginal_likelihood(data, low, (low + up) / 2, c, depth + 1, max_depth, qdist, aj),
            pt_marginal_likelihood(data, (low + up) / 2, up, c, depth + 1, max_depth, qdist, aj)
        ]    #likelihood of subpartitions
    else:
        likelihoods = [
            pt_marginal_likelihood(data, low, (low + up) / 2, c, depth + 1, max_depth, qdist, aj),
            pt_marginal_likelihood(data, (low + up) / 2, up, c, depth + 1, max_depth, qdist, aj),
            pt_marginal_likelihood(data, [low[0], (low[1] + up[1]) / 2], [(low[0] + up[0]) / 2, up[1]], 
                                   c, depth + 1, max_depth, qdist, aj),
            pt_marginal_likelihood(data, [(low[0] + up[0]) / 2, low[1]], [up[0], (low[1] + up[1]) / 2],
                                   c, depth + 1, max_depth, qdist, aj)
        ]    

    return logl + np.sum(likelihoods)

def lmbeta(*args):
    return np.sum(gammaln(args)) - gammaln(np.sum(args))

def is_discrete(X):
    return np.all(np.isin(X, np.arange(11)))    #evaluates to TRUE only if every element of X lies within the specified range of 0 to 10.

def scale(data):
    return (data - np.mean(data)) / np.std(data)


In [35]:
np.random.seed(0)
samplesize = 1000

mean1 = 8
sd1 = 10

mean2 = 16
sd2 = 12

data1 = np.random.normal(loc=mean1, scale=sd1, size=samplesize)
data2 = np.random.normal(loc=mean2, scale=sd2, size=samplesize)

'''
data1 = [20.62954285, 4.73766639, 21.29799263, 20.72429321, 12.14641434, -7.39950042, -1.28567035, 5.05279553, 7.94232827, 32.04653389,
    15.63593461, 0.00990751, -3.47657009, 5.10538426, 5.00784882, 3.88489167, 10.52223448, -0.91921127, 12.35683299, -4.37538422,
    5.75732115, 11.77395646, 9.33336361, 16.04189510, 7.42893226, 13.03607972, 18.85769362, 1.09046160, -4.84599354, 8.46726172,
    5.64293444, 2.57111745, 3.66689683, 1.50528353, 15.26750747, 19.51911754, 17.92160365, 3.70486891, 20.38304101, 5.20653718,
    25.57903090, 13.60746091, 3.47216027, -0.32043296, -3.66570547, -2.65590580, -7.63782051, 19.56536997, 16.32047129, 5.72671309
]

data2 = [
    19.1936483, 11.4795674, 45.2963755, 6.4559306, 15.3414703, 19.0016959, 23.4189195, 13.9285180, -10.6868033, 0.8366274,
    20.3047468, 15.8674543, 4.7122100, 14.6100961, 6.2203755, 18.9071618, -1.1011807, 20.3912935, 18.9809518, 16.7834582,
    16.2298767, 19.0880605, 8.2118791, 14.5699749, 23.9696284, 29.2116292, 17.7252578, 14.5869568, 5.0551796, -1.2510349,
    6.4349257, 31.0489973, 25.2657062, 13.3658125, 10.9022766, 10.9722388, 27.9638423, 12.6906637, 31.0722258, 23.7600927,
    31.5917476, 5.5208547, 16.1004515, 5.4295393, 23.1551082, 17.4366117, 12.6139135, 33.4718608, 18.7482351, 27.9585271
]

'''

pt_d_sample_test(data1, data2, log_BF=True)


{'bf': -0.061197476333745726, 'p_H0': None, 'p_H1': None}

In [33]:
len([0])

1